In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
tfs=transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.3337, 0.3064, 0.3171), ( 0.2672, 0.2564, 0.2629))
])

In [ ]:
trainset = torchvision.datasets.GTSRB(
    root='./root', split='train',  transform=tfs,download=True)
testset = torchvision.datasets.GTSRB(
    root='./root', split='test',  transform=tfs,download=True)

In [ ]:
print(len(trainset))
print(len(testset))

In [ ]:
#Hyper-parameters
epoch_count=20
batch_size=32


# for training:
learning_rate = 0.001
momentum = 0.9

In [ ]:
torch.manual_seed(143)
tst_size = 5000
val_size = len(testset) - tst_size
val_data, tst_data = torch.utils.data.random_split(testset, [val_size, tst_size])

ids=torch.randperm(len(testset))

split=tst_size
val_ids,test_ids=ids[split:],ids[:split]

test_sampler=SubsetRandomSampler(test_ids)
val_sampler=SubsetRandomSampler(val_ids)


#Dataloaders
test_loader = DataLoader(testset, batch_size, pin_memory=True,sampler=test_sampler)
val_loader = DataLoader(testset, batch_size,pin_memory=True,sampler=val_sampler)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True,pin_memory=True)

In [ ]:
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [ ]:
printsampler=DataLoader(testset,20,sampler=test_sampler)

def grid_plot(img):
    img=img/2 + .5
    print(img.shape)
    plt.figure(figsize=(15,4))
    plt.axis('off')
    plt.imshow(img.permute(1, 2, 0))
    #plt.savefig(f'grid.png')
    plt.show()

dataiter=iter(printsampler)
images,labels=next(dataiter)

grid_plot(torchvision.utils.make_grid(images))
print(labels)

In [ ]:
class ImgClassifier(nn.Module):
  def __init__(self):
      super(ImgClassifier, self).__init__()
      self.conv1 = nn.Conv2d(3,32,3)
      self.conv2 = nn.Conv2d(32,64,3)

      self.pool1 = nn.MaxPool2d(2)
      self.dout1 = nn.Dropout(0.5)
      self.conv3 = nn.Conv2d(64,512,3)
      self.conv4 = nn.Conv2d(512,512,3)

      self.pool2 = nn.MaxPool2d(2)
      self.dout2 = nn.Dropout(0.5)
      self.fc=nn.Linear(512*5*5,1024)
      self.out=nn.Linear(1024,43)

  def forward(self, x):

      x= self.conv1(x)
      x=F.elu(x)

      x= self.conv2(x)
      x=F.elu(x)

      x=self.pool1(x)
      x=self.dout1(x)

      x= self.conv3(x)
      x=F.elu(x)

      x= self.conv4(x)
      x=F.elu(x)

      x=self.pool2(x)
      x=self.dout2(x)
      #flatten all dimension except batch size
      x=torch.flatten(x,1)

      x = self.fc(x)
      x=F.elu(x)
      x=self.out(x)
      output=F.log_softmax(x,dim=1)

      return output

model=ImgClassifier()
model = model.to(device)  # put all model params on GPU.

# Create loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
print(model)

In [ ]:
foo1=[]#train_loss
foo2=[]#train_accuracy
foo3=[]#valid_loss
foo4=[]#valid_accuracy
bestmodel={'epoch':  0,
        'model_state_dict': model.state_dict(),
        'loss':   0,
        'accuracy':  0
        }

In [ ]:
for epoch in range(1,epoch_count+1):
    running_loss=0.0
    running_total = 0
    running_correct = 0
    run_step = 0

    for i, (images, labels) in enumerate(train_loader):
        model.train()

        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()  # reset gradients.
        loss.backward()  # compute gradients.
        optimizer.step()  # update parameters.

        running_loss += loss.item()
        running_total += labels.size(0)

        with torch.no_grad():
            _, predicted = outputs.max(1)
        running_correct += (predicted == labels).sum().item()
        run_step += 1
        if i %  500== 0:
            # check accuracy.
            print(f'epoch: {epoch}, steps: {i}, '
                  f'train_loss: {running_loss / run_step :.3f}, '
                  f'running_acc: {100 * running_correct / running_total:.1f} %')
            foo1.append(running_loss/run_step)
            foo2.append(100 * running_correct / running_total)
            running_loss = 0.0
            running_total = 0
            running_correct = 0
            run_step = 0

    # validation
    best_val_acc=[0,0]
    correct = 0
    val_loss=0
    total = 0
    counter=0
    model.eval()

    with torch.no_grad():
        for data in val_loader:
            counter+=1
            images,labels=data
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            val_loss += loss_fn(outputs,labels).item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_acc = 100 * correct / total

    if 100 * correct / total >= bestmodel['accuracy']:
      bestmodel={'epoch':  epoch,
        'model_state_dict': model.state_dict(),
        'loss':   val_loss,
        'accuracy':   val_acc
        }
    print(f'Validation accuracy: {100 * correct / total: .3f}%   ,'
              f'Validation loss:{val_loss/counter:.3f}')
    foo3.append(val_loss/counter)
    foo4.append(100 * correct / total)

print('Finished Training')

In [ ]:
lastmodel={'epoch':  epoch_count,
        'model_state_dict': model.state_dict(),
        'loss':   val_loss,
        'accuracy':   val_acc
        }
#torch.save(lastmodel,'lastmodel.pth')
#torch.save(bestmodel,'bestmodel.pth')

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    model.eval() # Set model in eval mode. Don’t forget!
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)

        outputs=model(images)
        _, predicted = outputs.max(dim=1)
        # predicted.shape: (B)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    test_acc = 100 * correct / total
    print(f'Test accuracy: {test_acc} %')
    print(f'Test error rate: {100 - 100 * correct / total: .2f} %')

In [ ]:

x=np.linspace(0,20,40)
x1=np.arange(0,20)

plt.figure(figsize=(10,7))
plt.plot(
    x,foo1,'g-',label='training loss'
)
plt.plot(
    x1,foo3,'b-',label='validation loss'
)
plt.xlabel("Epoch count")
plt.ylabel("Magnitude")
plt.title("Training and Validation Losses")
plt.legend()
plt.savefig('val_train_loss.png',dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(
    x,foo2,'r-',label='training accuracy'
)
plt.plot(
    x1,foo4,'b-',label='validation accuracy'
)
plt.xlabel("Epoch count")
plt.ylabel("Magnitude")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.savefig('val_train_acccuracy.png',dpi=300)
plt.show()